# Robustness Checks

In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib as mpl 

mpl.rcParams['figure.dpi']= 200
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

from rpy2.robjects import pandas2ri
pandas2ri.activate()

%reload_ext rpy2.ipython

In [2]:
%%R

library('lme4')
library('margins')
library("performance")
library('tidyverse')
library('broom.mixed')

getICCs <- function(m, type = 'poisson'){
  var_s <- as.numeric(getME(m, "theta")[1]^2) # seller level variance
  var_w <- as.numeric(getME(m, "theta")[2]^2) # week level variance
  
  if(type == 'poisson'){
    lambda = .139
    alpha <- log(1 + 1/lambda)
  }
  if(type == 'binomial'){
    alpha <- (pi^2) / 3
  }
  
  icc <- list(
    s = (var_s)/ (var_s + var_w + alpha),
    w = (var_w)/ (var_s + var_w + alpha),
    t = (var_s + var_w)/ (var_s + var_w + alpha),
    a = (alpha)/ (var_s + var_w + alpha)
  )
  
  return(icc)
}

sessionInfo()

R[write to console]: Loading required package: Matrix

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

R[write to console]: Registered S3 method overwritten by 'broom.mixed':
  method      from 
  tidy.gamlss broom



R version 4.0.3 (2020-10-10)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS  11.2.1

Matrix products: default
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] broom.mixed_0.2.6 forcats_0.5.0     stringr_1.4.0     dplyr_1.0.2      
 [5] purrr_0.3.4       readr_1.4.0       tidyr_1.1.2       tibble_3.0.4     
 [9] ggplot2_3.3.2     tidyverse_1.3.0   performance_0.5.1 margins_0.3.23   
[13] lme4_1.1-25       Matrix_1.2-18    

loaded via a namespace (and not attached):
 [1] statmod_1.4.35    TMB_1.7.19        tidyselect_1.1.0  reshape2_1.4.4   
 [5] splines_4.0.3     haven_2.3.1       lattice_0.20-41   colorspace_1.4-1 
 [9] vctrs_0.3.4       generics_0.0.2    blob_1.2.1        rlang_0.4.8      
[13] nloptr_1.2.2.2    pillar_1.4.6      withr_2.3.0       glue_1.4.2     

## Load Data

In [3]:
MAINDIR = os.getcwd().rsplit('/', 1)[0]
file = os.path.join(MAINDIR, 'data/analysis', 'vendor_item_period.csv')

In [4]:
df = pd.read_csv(file, index_col = False)
df = df.drop(df.columns[0], axis=1)

In [5]:
me = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'market_entry.pickle'))

## Robustness of ARF

In our paper we used liberal cut of values, here we provide robustness checks for stricter cut of criteria

In [6]:
# setting cut of values
t_cut = me['delta_t'].quantile(1/3)
p_cut = me['price_usd'].quantile(1/3)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(1/3)


# ARF 1
arf1 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_025')

# setting cut of values
t_cut = me['delta_t'].quantile(2/5)
p_cut = me['price_usd'].quantile(2/5)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(2/5)


# ARF 1
arf2 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_041')

arf = pd.concat([arf1, arf2], axis = 1).reset_index()\
    .assign(vendor = lambda x: x['vendor'].factorize()[0] + 1)

## Robustness of ARM

In our paper we used global cut of values to determine cases of artificial growth. However, some sellers might aswell have show fast, static and cheap growth by definition. Therefore we perform robustness checks were we includes a stricter definition of ARM, including internally defined cut-off values. This assumes that ARM is a strategy that is used at some time-point but not all the time during market activity. However, it assures that the growth of reputation is relatively fast, static and cheap for sellers themselves. 

In [7]:
# make ARM variable
t_cut_per_seller = df.groupby('vendor')['delta_t_m'].quantile(1/2).to_dict()
s_cut_per_seller = df.groupby('vendor')['delta_t_s'].quantile(1/2).to_dict()
p_cut_per_seller = df.groupby('vendor')['price_usd_m'].quantile(1/2).to_dict()
t_cut = df['delta_t_m'].quantile(1/2)
p_cut = df['price_usd_m'].quantile(1/2)
s_cut = df['delta_t_s'].quantile(1/2)


arm = df\
    .assign(t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()

arm['arm_t__ps'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_p__ts'] = df\
    .assign(t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low', 'p_low_s', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_s__tp'] = df\
    .assign(t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller))\
    .assign(arm = lambda x: (x[['t_low', 'p_low', 's_low_s']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tp__s'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s',  's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_ts__p'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            p_low   = lambda x: x['price_usd_m'] < p_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 's_low_s', 'p_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_ps__t'] = df\
    .assign(p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut)\
    .assign(arm = lambda x: (x[['p_low_s', 's_low_s', 't_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']



# Shift Dependent variables with one week 
for col in ['arm', 'arm_t__ps', 'arm_p__ts', 'arm_s__tp',
            'arm_tp__s', 'arm_ts__p', 'arm_ps__t']: 
    arm['{0}_shift'.format(col)] = arm\
        .groupby('vendor')\
        .apply(lambda x: x[col].shift(1))\
        .reset_index(drop = True)

In [8]:
# import person week file
df = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'vendor_week.pickle'))
df = df.merge(df.groupby('vendor')['me_min'].min().rename('me'), on = 'vendor')
df = df.rename(columns= {
    'international_shipment_count_w': 'int_shipment_count_w', 
    'international_shipment': 'int_shipment'})

In [9]:
df = df.merge(arm, on = ['vendor', 'w']).merge(arf, on = ['vendor'])

In [10]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

# normalize variables
skewed_variables = [
    'neg_count_min', 'pos_count_min', 'neg_count_w', 
    'neg_count_w_shift', 'pos_count_w', 'pos_count_w_shift', 'sales_volume_w', 
    'item_count_w', 'int_shipment_count_w']

bc_vars = pd.DataFrame(
    pt.fit_transform(df[skewed_variables]),
    columns = ['bc_' + var for var in skewed_variables])

c_vars = df[skewed_variables].add_prefix('c_')

dummy_vars = df.assign(
        items   = pd.qcut(df['item_count'], 3,  labels=["low", "medium", "high"]),
        items_w = pd.qcut(df['item_count_w'], 3,  labels=["low", "medium", "high"]),
        sales   = pd.qcut(df['cum_count'], 2,  labels=["low", "high"]),
        sales_w = pd.qcut(df['sales_volume_w'], 2,  labels=["low", "high"]))\
    .loc[:,['items', 'items_w', 'sales', 'sales_w']]

n_vars = df[['cum_count', 'sales_volume_w', 'item_count', 
             'item_count_w', 'me', 'int_shipment_count_w',
             'empty_stock_last_week_count_w', 'empty_stock_last_week_maxw']].rename({'cum_count': 'sales_volume'})

bool_vars = df[
    ['arf', 'arf_025', 'arf_041', 'arm_maxw', 'arm_maxw_shift', 
     'arm', 'arm_t__ps', 'arm_p__ts', 'arm_s__tp',
     'arm_tp__s', 'arm_ts__p', 'arm_ps__t',
     'int_shipment', 'has_price_drop', 'has_price_drop_shift']].astype('int')

lev_vars =  df[['vendor','w', 'me_min']]

data = pd.concat([lev_vars, bool_vars, bc_vars, c_vars, dummy_vars, n_vars], axis = 1)

In [11]:
%%R -i data

data['me_'] <- scale(data$me)[,1]
data['w_'] <- scale(data$w)[,1]
data['me_2'] <- scale(data$me^2)[,1]
data['w_2'] <- scale(data$w^2)[,1]

variables = c(
    'bc_pos_count_w', 'bc_item_count_w', 'bc_int_shipment_count_w',
    'bc_neg_count_w', 'c_neg_count_w', 'c_pos_count_w')

for (var in variables){
  for (vendor in unique(data$vendor)){
    # calculate mean and deviances
    x = data[data$vendor == vendor, var]
    m = mean(x)
    dev = x - m
    
    # concat information to dataframe
    data[data$vendor == vendor, paste(var, "m", sep=".")] <- m
    data[data$vendor == vendor, paste(var, "dev", sep=".")] <- dev
  }
}

Optimizer <- glmerControl(optimizer = "bobyqa",
                          optCtrl = list(maxfun=2e5))

model_arf <- list()

## ARF
### Stricter criteria

In [12]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a114 = model))

In [13]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a041 = model))

In [14]:
%%R
model <- glmer(c_neg_count_w_shift ~ arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

model_arf <- append(model_arf, list(neg_a025 = model));

In [15]:
%%R
model <- glmer(arm ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a114 = model))

In [16]:
%%R
model <- glmer(arm ~ arf_041
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_041
      + bc_neg_count_w.dev:arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a041 = model))

In [17]:
%%R
model <- glmer(arm ~ arf_025
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_025
      + bc_neg_count_w.dev:arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arf <- append(model_arf, list(arm_a025 = model))

In [18]:
%%R -o results

cols = c("B", 'se', 'z', 'p')

results = data.frame()

for (name in names(model_arf)){
    model <- round(data.frame(summary(model_arf[[name]])[10]),3)
    colnames(model) <- cols
    rownames(model) <- paste(name, rownames(model), sep = '__')
    
    odds <- data.frame(tidy(model_arf[[name]],conf.int=TRUE,exponentiate=TRUE,effects="fixed")) %>%
        select(c("estimate", "conf.low", "conf.high")) %>%
        mutate_all(funs(round(.,3)))
    colnames(odds) <- c("or", "ci_low", "ci_high")
    
    model <- cbind(model, odds)
    
    results <- rbind(results, model)
}

In [19]:
def sig_stars(x):
    if x < .001: 
        return "***"
    if x < .01: 
        return "**"
    if x < .05:
        return "*"
    if x < .1: 
        return "."
    else: 
        return ""

ci_fmt = lambda x: f"({str(round(x['ci_low'], 3)).ljust(5, '0')}; {str(round(x['ci_high'], 3)).ljust(5, '0')})"
order = [0, 1] + [12, 13, 14, 15] + [ 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
col_order = [5, 4, 3, 2, 1, 0]

r = results\
    .assign(star = lambda x: x['p'].apply(sig_stars))\
    .assign(_or = lambda x: x["or"].apply(lambda v: f'{v:.3f}') + x["star"],
            ci = lambda row: row.agg(ci_fmt, axis=1))[['_or', 'ci']]\
    .reset_index()\
    .melt(id_vars = 'index')\
    .assign(
        condition = lambda x: x['index'].str.split('__').str[0],
        coefficient = lambda x: x['index'].str.split('__').str[1]\
                                          .str.replace('_041', '')\
                                          .str.replace('_025', ''))\
    .assign(
        coefficient = lambda x: pd.Categorical(x['coefficient'], 
                                               ordered=True, 
                                               categories=[x['coefficient'].unique()[i] for i in order]))\
    .drop('index', axis = 1)\
    .set_index(['coefficient', 'variable', 'condition'])\
    .unstack()[lambda x: [x.columns[i] for i in col_order]]

r

value                     \
condition                                      neg_a114           neg_a041   
coefficient                 variable                                         
(Intercept)                 _or                 0.631**            0.660**   
                            ci           (0.470; 0.848)     (0.493; 0.885)   
arf                         _or                 1.606**             2.008*   
                            ci           (1.194; 2.160)     (1.076; 3.749)   
bc_neg_count_w.m            _or                     NaN                NaN   
                            ci                      NaN                NaN   
bc_neg_count_w.dev          _or                     NaN                NaN   
                            ci                      NaN                NaN   
arf:bc_neg_count_w.m        _or                     NaN                NaN   
                            ci                      NaN                NaN   
arf:bc_neg_count_w.dev      _or                     NaN                NaN   
                            ci                      NaN                NaN   
bc_pos_count_w.m            _or                   1.186             1.240.   
                            ci           (0.954; 1.475)     (0.999; 1.538)   
bc_pos_count_w.dev          _or                1.043***           1.043***   
                            ci           (1.032; 1.054)     (1.032; 1.054)   
bc_item_count_w.m           _or                1.847***           1.766***   
                            ci           (1.479; 2.306)     (1.414; 2.204)   
bc_item_count_w.dev         _or                0.848***           0.848***   
                            ci           (0.826; 0.869)     (0.826; 0.869)   
bc_int_shipment_count_w.m   _or                1.498***           1.515***   
                            ci           (1.319; 1.702)     (1.333; 1.722)   
bc_int_shipment_count_w.dev _or                1.137***           1.137***   
                            ci           (1.091; 1.185)     (1.091; 1.185)   
me_                         _or                  1.293.             1.282.   
                            ci           (0.998; 1.675)     (0.989; 1.662)   
me_2                        _or                0.638***           0.639***   
                            ci           (0.503; 0.809)     (0.504; 0.811)   
w_                          _or               54.618***          54.582***   
                            ci        (18.842; 158.322)  (19.362; 153.866)   
w_2                         _or                0.006***           0.006***   
                            ci           (0.002; 0.018)     (0.002; 0.017)   

                                                                            \
condition                                      neg_a025           arm_a114   
coefficient                 variable                                         
(Intercept)                 _or                 0.673**           0.002***   
                            ci           (0.502; 0.901)     (0.002; 0.003)   
arf                         _or                   1.069            1.923**   
                            ci           (0.463; 2.472)     (1.199; 3.084)   
bc_neg_count_w.m            _or                     NaN           2.433***   
                            ci                      NaN     (1.840; 3.218)   
bc_neg_count_w.dev          _or                     NaN              1.010   
                            ci                      NaN     (0.930; 1.097)   
arf:bc_neg_count_w.m        _or                     NaN              1.007   
                            ci                      NaN     (0.522; 1.939)   
arf:bc_neg_count_w.dev      _or                     NaN              1.054   
                            ci                      NaN     (0.882; 1.259)   
bc_pos_count_w.m            _or                  1.254*          21.737***   
                            ci           (1.010; 1.557)   (14.566; 32.4

## ARM
### Internal and external cut-offs

In [20]:
%%R

model_arm <- list()

model <- glmer(arm ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm = model))

In [21]:
%%R

model <- glmer(arm_t__ps ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_t__ps = model))

In [22]:
%%R

model <- glmer(arm_p__ts ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_p__ts = model))

In [23]:
%%R

model <- glmer(arm_s__tp ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_s__tp = model))

In [24]:
%%R
model <- glmer(arm_tp__s ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_tp__s = model))

In [25]:
%%R
model <- glmer(arm_ts__p ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_ts__p = model))

In [29]:
%%R
model <- glmer(arm_ps__t ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

model_arm <- append(model_arm, list(arm_ps__t = model))

In [30]:
%%R -o results2

cols <- c("B", 'se', 'z', 'p')

results2 <- data.frame()

for (name in names(model_arm)){
    model <- round(data.frame(summary(model_arm[[name]])[10]),3)
    colnames(model) <- cols
    rownames(model) <- paste(name, rownames(model), sep = '___')

    odds <- data.frame(tidy(model_arm[[name]],conf.int=TRUE,exponentiate=TRUE,effects="fixed")) %>%
        select(c("estimate", "conf.low", "conf.high")) %>%
        mutate_all(funs(round(.,3)))
    colnames(odds) <- c("or", "ci_low", "ci_high")
    
    model <- cbind(model, odds)
     
    results2 <- rbind(results2, model)
}


In [31]:
def sig_stars(x):
    if x < .001: 
        return "***"
    if x < .01: 
        return "**"
    if x < .05:
        return "*"
    if x < .1: 
        return "."
    else: 
        return ""

order = [0, 1, 2, 3, 4]

results2\
    .assign(star = lambda x: x['p'].apply(sig_stars))\
    .assign(_or = lambda x: x["or"].apply(lambda v: f'{v:.3f}') + x["star"],
            ci = lambda row: row.agg(ci_fmt, axis=1))[['_or', 'ci']]\
    .reset_index()\
    .melt(id_vars = 'index')\
    .assign(
        condition = lambda x: x['index'].str.split('___').str[0],
        coefficient = lambda x: x['index'].str.split('___').str[1]\
                                          .str.replace('_041', '')\
                                          .str.replace('_025', ''))\
    .drop('index', axis = 1)\
    .loc[lambda x: (x['coefficient'].str.contains('arf')) | (x['coefficient'].str.contains('neg_count'))]\
    .assign(
        coefficient = lambda x: pd.Categorical(x['coefficient'], 
                                               ordered=True, 
                                               categories=[x['coefficient'].unique()[i] for i in order]))\
    .sort_values('coefficient')\
    .pivot_table(index= 'coefficient', columns= ['condition', 'variable'], values = 'value', aggfunc='max').T

coefficient                    arf bc_neg_count_w.m bc_neg_count_w.dev  \
condition variable                                                       
arm       _or              1.923**         2.433***              1.010   
          ci        (1.199; 3.084)   (1.840; 3.218)     (0.930; 1.097)   
arm_p__ts _or                1.045         1.398***              0.988   
          ci        (0.784; 1.394)   (1.177; 1.661)     (0.910; 1.073)   
arm_ps__t _or                1.015         1.428***              0.978   
          ci        (0.747; 1.379)   (1.186; 1.719)     (0.904; 1.059)   
arm_s__tp _or              2.077**         2.557***              1.015   
          ci        (1.276; 3.381)   (1.908; 3.428)     (0.938; 1.099)   
arm_t__ps _or              1.550**         1.412***              0.994   
          ci        (1.162; 2.068)   (1.189; 1.677)     (0.945; 1.046)   
arm_tp__s _or                0.996           1.132*              1.020   
          ci        (0.818; 1.215)   (1.003; 1.278)     (0.970; 1.072)   
arm_ts__p _or               1.518*         1.854***              1.001   
          ci        (1.017; 2.266)   (1.464; 2.348)     (0.952; 1.052)   

coefficient        arf:bc_neg_count_w.m arf:bc_neg_count_w.dev  
condition variable                                              
arm       _or                     1.007                  1.054  
          ci             (0.522; 1.939)         (0.882; 1.259)  
arm_p__ts _or                    1.469.                 1.184.  
          ci             (0.981; 2.198)         (0.990; 1.416)  
arm_ps__t _or                    1.581*                  1.119  
          ci             (1.027; 2.433)         (0.947; 1.322)  
arm_s__tp _or                     0.977                  1.037  
          ci             (0.494; 1.933)         (0.878; 1.224)  
arm_t__ps _or                    0.638*                  1.016  
          ci             (0.420; 0.971)         (0.899; 1.147)  
arm_tp__s _or                     0.994                  0.996  
          ci             (0.744; 1.328)         (0.882; 1.125)  
arm_ts__p _or                   0.425**                  1.005  
          ci             (0.238; 0.760)         (0.895; 1.129)